# Mock data
This notebook prepares some mock data to be used in the GUI demonstrator>

In [1]:
from data_schemas import BatterySpecs, BatteryComposition, BatteryCapacity, BatteryMetadata, BatterySample
import datetime
import json, random
import pandas as pd
import numpy as np
import names

In [2]:
STD_SPECS = pd.read_csv('sample_specs.csv', dtype=str)
STD_SPECS

,manufacturer,composition,capacity,form_factor
0,BIG-MAP,C | LP57 | NMC811,1.539 mAh,2032
1,BIG-MAP,C | LP57 | LNO,1.539 mAh,2032
2,BIG-MAP,Si-C | LP57 | NMC811,1.539 mAh,2032
3,BIG-MAP,Si-C | LNMO | NMC622,1.539 mAh,2032
4,BIG-MAP,C | LP57 | NMC811,4.618 mAh,2032
5,BIG-MAP,C | LP57 | LNO,4.618 mAh,2032
6,BIG-MAP,Si-C | LP57 | NMC811,4.618 mAh,2032
7,BIG-MAP,Si-C | LNMO | NMC622,4.618 mAh,2032
8,Conrad energy,Li-based,45 mAh,2032
9,Conrad energy,Li-based,60 mAh,2032


In [3]:
STD_SPECS.keys()

Index(['manufacturer', 'composition', 'capacity', 'form_factor'], dtype='object')

In [4]:
for key in STD_SPECS.keys():
    print(f'{key:14}:  {STD_SPECS.get(key).unique().tolist()}')

manufacturer  :  ['BIG-MAP', 'Conrad energy', 'Maxell']
composition   :  ['C | LP57 | NMC811', 'C | LP57 | LNO', 'Si-C | LP57 | NMC811', 'Si-C | LNMO | NMC622', 'Li-based', 'LiMgO2-based']
capacity      :  ['1.539 mAh', '4.618 mAh', '45 mAh', '60 mAh']
form_factor   :  ['2032', '2016']


In [114]:
def random_datetime(
        start=datetime.datetime(2020, 1, 1, 0, 0, 0, tzinfo=datetime.timezone.utc),
        end=datetime.datetime.now(tz=datetime.timezone.utc)):
    """Generate a random datetime between `start` and `end`"""
    return start + datetime.timedelta(
        # Get a random amount of seconds between `start` and `end`
        seconds=random.randint(0, int((end - start).total_seconds())))

In [115]:
def create_random_sample():
    # pick one of the available specs
    random_spec = STD_SPECS.sample().iloc[0]
    battery_id = random.randint(0, 1000)
    sample = BatterySample(
        manufacturer = random_spec.manufacturer,
        composition = BatteryComposition(description=random_spec.composition),
        form_factor = random_spec.form_factor,
        capacity = BatteryCapacity(nominal=float(random_spec.capacity.split()[0]), units=random_spec.capacity.split()[1]),
        metadata = BatteryMetadata(
            creation_datetime=random_datetime(),
            creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'
        ),
        battery_id = battery_id,
        name = f'{names.get_first_name()}_{battery_id}'
    )
    return sample

Now we generate 20 random samples and save the to a JSON file.

In [128]:
samples = [create_random_sample() for _ in range(20)]
samples = sorted(samples, key=lambda s: s.battery_id)

In [130]:
class BatterySampleEncoder(json.JSONEncoder):
        def default(self, obj):
            if isinstance(obj, datetime.datetime):
                return obj.isoformat()
            else:
                super().default(obj)

json.dump([s.dict() for s in samples], open('available_samples.json', 'w'), cls=BatterySampleEncoder)
json.dump({s.battery_id: s.dict() for s in samples}, open('available_samples_id.json', 'w'), cls=BatterySampleEncoder)

In [129]:
[s.battery_id for s in samples]

[23,
 35,
 36,
 47,
 98,
 225,
 271,
 336,
 405,
 524,
 588,
 609,
 727,
 748,
 788,
 809,
 834,
 859,
 896,
 948]

Reload

In [131]:
data = json.load(open('available_samples.json', 'r'))
new_samples = [BatterySample.parse_obj(dic) for dic in data]

In [132]:
s = new_samples[0]
s

BatterySample(manufacturer='BIG-MAP', composition=BatteryComposition(description='Si-C | LP57 | NMC811', cathode='Si-C', anode='NMC811', electrolyte='LP57'), form_factor='2032', capacity=BatteryCapacity(nominal=4.618, actual=None, units='mAh'), metadata=BatteryMetadata(creation_datetime=datetime.datetime(2021, 12, 11, 18, 38, 24, tzinfo=datetime.timezone.utc), creation_process='Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed mollis arcu ac risus vestibulum, ac commodo nunc euismod. Vivamus suscipit eros nec justo faucibus, et eleifend diam eleifend.'), battery_id=23, name='Robert_23')